In [1]:
def findroutes(filename, limit_data=0, driver_id=0, prints=False):    
    import json
    import math
    import numpy as np
    import warnings
    import matplotlib.pyplot as plt
    import random
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    from sklearn.preprocessing import MinMaxScaler

    def load_json(filename):
        with open(filename, 'r') as file:
            data4 = json.load(file)
        return data4

    def load_data(actual_routes_file1, limit_actual_routes1=0, driver_id=0):
        #load data and limit it as required
        
        #load driverdata
        driver_data1=load_json(actual_routes_file1)

        #limit data to only the driver that is selected
        if driver_id!=0:
            driver_data=[]
            for route in driver_data1:
                if route["driver"]==driver_id:
                    driver_data.append(route)
            driver_data1=driver_data

        #if a limit on total routes is set apply it
        if limit_actual_routes1!=0:
            driver_data1=driver_data1[:limit_actual_routes1]
        return driver_data1

    def find_numb_dim_and_conn(data):
        #find the amount of dimensions (possible merch) for all the possible 
        #connections between cities 
        dim_count={}
        standardroutes=set()
        for route_info in data:
            standardroutes.add(route_info["sroute"])
            for trip_info in route_info["route"]:
                
                set_of_items=set()
                
                for merchandise in trip_info["merchandise"]:
                    set_of_items.add(merchandise)

                conn_name=trip_info["from"]+"-"+trip_info["to"]
                
                if conn_name not in dim_count:
                    dim_count[conn_name]=set_of_items
                else:
                    dim_count[conn_name].update(set_of_items)

        #create a mapping method by converting the sets (first for speed) into
        #tuples which take less memory than lists
        #we do this so we can say that 'pens' is for example the first dimension
        #and 'milk' the second
        for conn_name in dim_count:
            dim_count[conn_name]=(tuple(dim_count[conn_name]))
        
        return dim_count, len(standardroutes)

    def createpoints(data, dim_count):   
        #convert the data into lists of data points so the clustering can be applied
        data_points={}
        for conn_name in dim_count:
            data_points[conn_name]=[]

        for route_info in data:
            for trip_info in route_info["route"]:
                conn_name=trip_info["from"]+"-"+trip_info["to"]
                
                temp_point=[0] * len(dim_count[conn_name])
                
                for merch in trip_info["merchandise"]:
                    index=dim_count[conn_name].index(merch)
                    temp_point[index]=trip_info["merchandise"][merch]
                            
                data_points[conn_name].append(temp_point)
        
        return data_points

    def clustering(data_points):
        #find all clusters

        #ignore warnings temporarily for better readability
        warnings.filterwarnings("ignore")

        def sample(upper_limit1, amount_of_samples1, data1):
            rand_numbs=random.sample(range(0, upper_limit1), amount_of_samples1)
            sample_space=[list(data1[i]) for i in rand_numbs]
            return sample_space

        def round_list(lst, decimal_places=0):
            rounded_list = [round(element, decimal_places) for element in lst]
            return rounded_list

        ext_data_points={}
        clusterinfo={}
        count=0
        for city in data_points:
            count=count+1
            max_expected_clusters=0
            labeling={}
            data = data_points[city]
            #limit the sample space
            datasize=len(data)
            upper_limit=datasize
            if datasize<1000:
                amount_of_samples=min(datasize, 100)
                max_expected_clusters=round(float(amount_of_samples)**(1/2))
            elif datasize<10000:
                amount_of_samples=round(datasize/10)
                max_expected_clusters=round(float(amount_of_samples)**(1/2))

            else:
                amount_of_samples=round(datasize/100)
                max_expected_clusters=round(float(amount_of_samples)**(1/2))

            sample_space=[]
            
            #check if all the samplepoint    print(amount_of_samples)
            #are the same and if they are try again 5 times
            #if still the same pass the knowledge on
            for _ in range(5):
                one_point_marker=True
                sample_space=sample(upper_limit, amount_of_samples, data)
                if all(x == data[0] for x in sample_space) != True:
                    one_point_marker=False
                    break
                    
            # Calculate silhouette scores for different values of k using samples 
            silhouette_scores = []
            #limit expected amount of clusters proportional to the amount of datapoints we can maybe say that for every 2% of dataset there may be a cluster existing
            
            K_range = range(2, max_expected_clusters)


            
            if one_point_marker==False and max_expected_clusters>2:
                #loop through possible amount of clusters and stop for first silhouette with score >0.7
                for k in K_range:
                    try:
                        scaler = MinMaxScaler()
                        sample_space = scaler.fit_transform(sample_space)
                        kmeans = KMeans(n_clusters=k)
                        kmeans.fit(sample_space)
                        labels = kmeans.labels_
                        silhouette_scores.append(silhouette_score(sample_space, labels))
                    except ConvergenceWarning as e:
                        break
                    if max(silhouette_scores)>0.7:
                        break
                
                #determine k using the sampled space
                max_ss=max(silhouette_scores)
                if max_ss>0.6:
                    k=silhouette_scores.index(max_ss)+2
                else:
                    k=1
                    
            else:
                k=1
            #now use kmeans to find the labels of the points for the entire set
            scaler = MinMaxScaler()
            data = scaler.fit_transform(data)
            kmeans = KMeans(n_clusters=k)
            kmeans.fit(data)

            #build the extended data points set which also contain the labels of the points
            labels=kmeans.labels_
            
            for index, point in enumerate(data_points[city]):
                labeling[tuple(point)]=labels[index]
            ext_data_points[city]=labeling
            
            #build a dataset which contains info about each cluster
            for i in range(k):
                temp={}
                oneclusterdata=data[labels==i]
                temp["count"]=(len(oneclusterdata))
                kmeans1=KMeans(n_clusters=1)
                kmeans1.fit(oneclusterdata)
                temp["inertia"]=(kmeans1.inertia_)
                temp["centroid"]=tuple(round_list(scaler.inverse_transform(kmeans.cluster_centers_)[i]))
                clusterinfo[city+"-"+str(i)]=temp
        
        return clusterinfo, ext_data_points

    def build_dataset(data, dim_count, labeled_data_points):
        #Now build the new, transformed dataset out of the old dataset and the gathered data
        #the data we gathered are the mapping tool (dim_count) and the ext_data_points
        dataset=[]
        dim_map=dim_count
        clust_map=labeled_data_points

        for route_info in data:
            conv_route=[]
            cont_trips=[]
            
            for trip_info in route_info["route"]:
                conn_name=trip_info["from"]+"-"+trip_info["to"]
                
                temp_point=[0] * len(dim_map[conn_name])
                
                for merch in trip_info["merchandise"]:
                    index=dim_map[conn_name].index(merch)
                    temp_point[index]=trip_info["merchandise"][merch]
                cluster=clust_map[conn_name][tuple(temp_point)]
                trip_name=conn_name+"-"+str(cluster)
                cont_trips.append(trip_name)
                
            conv_route.append(route_info["id"])
            conv_route.append(route_info["driver"])
            conv_route.append(route_info["sroute"])
            conv_route.append(tuple(cont_trips))
            dataset.append(tuple(conv_route))
            
        return tuple(dataset)

    def find_freq_routes(dataset,dim_map ,NS):

        def FREQUENT_ITEMS(dataset,thresehold):
            def subsequence(m,S):
                #this function tells us wether a given sequence is a subsequence of a longer one.
                #basically we are certifying if a given subroute is part of a bigger route in the dataset.
                l=len(m)
                L=len(S)
                for i in range(0,L-l+1):
                    if S[i:i+l]==m:
                        return True
                return False
            
            def match(x,y):
                #A function that, given two trips, outputs wether they could be chained or not,
                #for example, if the first trip ends in Milano, it tells you wether the second one
                #starts in Milano too. In this way, we only create "possible couples" with the next function.
                w1=x
                l1=len(w1)
                w1=w1[:l1-2]
                index1=w1.index('-')
                w1=w1[index1+1:]
                
                w2=y
                l2=len(w2)
                w2=w2[:l2-2]
                index2=w2.index('-')
                w2=w2[:index2]
                
                return w1==w2 
        
            def prune(X,thresehold):
                #remove the singletons not frequent enough, depending on a certain thresehold. 
                #In this case, I've chosen 100. 
                #The function outputs a tuple with the frequent items and not frequently enough items.
                
                reduce=[]
                for i in X:
                    if X[i]<thresehold:
                        reduce.append(((i,),X[i]))
                for i in reduce:
                    X.pop(i[0][0])
                return (X,reduce)
            
            def couples(X):
                #using the above function we create couples.It also gives us a list 
                #of the singletons that we could not extend, and a list of the ones we could.
                #We need the latter in case the couples don't pass the prune in the next step;
                #in that case, we would go back one step and offer the routes that created that couple.

                nextcandidates=[]
                extend=[]
                for i in X:
                    extend.append(((i,),X[i]))
                lista=[]
                for i in X:
                    for j in X:
                        if match(i,j):
                            if ((i,),X[i]) in extend:
                                extend.remove(((i,),X[i]))
                            if ((j,),X[j]) in extend:
                                extend.remove(((j,),X[j]))
                                
                            if (i,j) not in lista:
                                lista.append((i,j))
                            if ((i,),X[i]) not in nextcandidates:
                                    nextcandidates.append(((i,),X[i]))
                            if ((j,),X[j]) not in nextcandidates:
                                    nextcandidates.append(((j,),X[j]))
                return (lista,extend,nextcandidates)
        
            def combine(X):
                #this function takes a list of tuples of the same length and combines them
                #to create new tuples of length + 1. Furthermore, it only creates "logical"
                #tuples. That is, it won't combine (1,2,3) with (1,2,5) but it will combine 
                #(1,2,3) with (2,3,4) to create (1,2,3,4). It also gives as a list of tuples
                #we could not extend, and the ones we could, for analogous reasons as in the 
                #case of the function couples.
                
                extend=[]
                for i in X:
                    extend.append((i,X[i]))
                nextcandidates=[]
                lista=[]
                if len(X)!=0:
                    for i in X:
                        l=len(i)
                        break
                    for tuple1 in X:
                        for tuple2 in X:
                            if tuple2[:l-1] == tuple1[1:]:
                                newtuple=tuple1 + (tuple2[l-1],)
                                if (tuple1,X[tuple1]) in extend:
                                    extend.remove((tuple1,X[tuple1]))
                                if (tuple2,X[tuple2]) in extend:
                                    extend.remove((tuple2,X[tuple2]))
                                if newtuple not in lista:
                                    lista.append(newtuple)
                                if (tuple1,X[tuple1]) not in nextcandidates:
                                    nextcandidates.append((tuple1,X[tuple1]))
                                if (tuple2,X[tuple2]) not in nextcandidates:
                                    nextcandidates.append((tuple2,X[tuple2]))
                return (lista,extend,nextcandidates)
        
            def find(candidates,X):
                #given possible frequent tuples it creates a dictionary with the occurrences
                #of such tuples as subsequences in the dataset.

                freq={}
                for i in candidates:
                    for j in X:
                        if subsequence(i,j[3]):
                            if i in freq:
                                freq[i]=freq[i]+1
                            else:
                                freq[i]=1
                return freq
            
            def offeroutes(OFFER):
                #It takes all of the cluster-routes and convert them into actual routes.

                def cluster_to_info(x,clusterinfo,dim_map):
                    #It takes a trip-clusters and converts it into an actual trip.
                    
                    w1=x
                    index1=w1.index('-')
                    w1=w1[index1+1:]
                    index3=w1.index('-')
                    w1=w1[:index3]
                    
                    w2=x
                    index2=w2.index('-')
                    w2=w2[:index2]
                    
                    trip={}
                    
                    trip['from']=w2
                    trip['to']=w1
                    
                    info=clusterinfo[x]['centroid']
            
                    merch=dim_map[w2+"-"+w1]
                    s={}
                    for i in range(len(merch)):
                        if info[i]!=0:
                            s[merch[i]] = info[i]
                            
                    trip['merchandise']=s

                    
                    return trip

                lista=[]
                H=len(OFFER)
                for i in range(H):
                    ruta={}
                    ruta['id']='s'+str(i+1)
                    TRIPS=[]
                    for j in OFFER[i][0]:
                        trip= cluster_to_info(j,clusterinfo,dim_map)
                        TRIPS.append(trip)
                    ruta['route']=TRIPS
                    lista.append((ruta,OFFER[i][1]))
                L=[]
                for i in lista:
                    L.append(i[1])
                L.sort()
                L=L[::-1]
                Lista=[]
                for i in L:
                    for j in lista:
                        if i==j[1]:
                            Lista.append(j[0])
                            lista.remove(j)
                            break   
                for i in range(H):
                    Lista[i]['id']='s'+str(i+1)
                return Lista
            
            OFFER=[]    
            freq={}
            for x in dataset:
                for i in x[3]:
                    if i not in freq:
                        freq[i] = 1
                    else:
                        freq[i] = freq[i] +1

            #we can also use clusterinfo to get freq
            #freq={}
            #for i in clusterinfo:
            #    freq[i]=clusterinfo[i]['count']
            
            S = prune(freq,thresehold)[0]
            (candidates,extend,nextcandidates)=couples(S)
            OFFER=OFFER + extend
            
            new={}
            while len(candidates)!=0:
                new=find(candidates,dataset)
                (S,H) = prune(new,thresehold)
                count=0
                for j in nextcandidates:
                    for i in H:
                        if subsequence(j[0],i[0]):
                            OFFER=OFFER + [j]
                            count=count+1
                    if count==0:
                        for i in S:
                            if subsequence(j[0],i):
                                count=count+1
                    if count==0:
                        OFFER=OFFER + [j]
                        
                (candidates,extend,nextcandidates)=combine(S)
                OFFER = OFFER+ extend
            return offeroutes(OFFER)
            
        #assumes that there are at least possible routes as there are standard routes
        n=math.floor(math.log2(NS))
        
        OFFER= FREQUENT_ITEMS(dataset,len(dataset)/(2**n))
        while len(OFFER)<NS*2:
            n=n+1
            OFFER=FREQUENT_ITEMS(dataset,len(dataset)/(2**n))
        return (OFFER[:NS])


    if (prints==True): print("Loading data")
    data = load_data(filename, limit_data, driver_id)

    if (prints==True): print("Finding dimension sizes")
    [dim_count, number_sroutes]=find_numb_dim_and_conn(data)

    if (prints==True): print("Creating points")
    data_points=createpoints(data, dim_count)

    if (prints==True): print("Clustering points")
    [clusterinfo, labels]=clustering(data_points)

    if (prints==True): print("Rebuild dataset")
    dataset=build_dataset(data, dim_count, labels)

    if (prints==True): print("Finding frequent routes")
    result=find_freq_routes(dataset, dim_count, number_sroutes)
    
    return result

In [3]:
filename="/home/felix/Documents/Python/DataMining/DataMining/data/actual.json"
result=findroutes(filename, 100, "C" , prints=True)
print(result)

Loading data
Finding dimension sizes
Creating points
Clustering points
Rebuild dataset
Finding frequent routes
[{'id': 's1', 'route': [{'from': 'Milano', 'to': 'Genova', 'merchandise': {'Meat': 22.0, 'Pasta': 25.0, 'Tea': 16.0}}, {'from': 'Genova', 'to': 'Siena', 'merchandise': {'Meat': 12.0, 'Butter': 3.0, 'Bread': 8.0, 'Pens': 17.0, 'Pasta': 27.0, 'Honey': 11.0}}, {'from': 'Siena', 'to': 'Trento', 'merchandise': {'Apples': 20.0, 'Rice': 3.0, 'Carrots': 6.0, 'Tomatoes': 6.0, 'Honey': 25.0}}]}]
